In [12]:
import pandas as pd

In [33]:
providers_df = pd.read_csv('NH_ProviderInfo_Jan2022.csv')
providers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15238 entries, 0 to 15237
Data columns (total 96 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Federal Provider Number                                                15238 non-null  object 
 1   Provider Name                                                          15238 non-null  object 
 2   Provider Address                                                       15238 non-null  object 
 3   Provider City                                                          15238 non-null  object 
 4   Provider State                                                         15238 non-null  object 
 5   Provider Zip Code                                                      15238 non-null  int64  
 6   Provider Phone Number                                                  15238 non-null 

In [37]:
state_averages_df = pd.read_csv('NH_StateUSAverages_Jan2022.csv')
state_averages_df.head()

,State or Nation,Cycle 1 Total Number of Health Deficiencies,Cycle 1 Total Number of Fire Safety Deficiencies,Cycle 2 Total Number of Health Deficiencies,Cycle 2 Total Number of Fire Safety Deficiencies,Cycle 3 Total Number of Health Deficiencies,Cycle 3 Total Number of Fire Safety Deficiencies,Average Number of Residents per Day,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,...,Percentage of long stay residents who received an antianxiety or hypnotic medication,Percentage of high risk long stay residents with pressure ulcers,Percentage of long stay residents assessed and appropriately given the seasonal influenza vaccine,Percentage of short stay residents who made improvements in function,Percentage of short stay residents who were assessed and appropriately given the seasonal influenza vaccine,Percentage of short stay residents who were rehospitalized after a nursing home admission,Percentage of short stay residents who had an outpatient emergency department visit,Number of hospitalizations per 1000 long-stay resident days,Number of outpatient emergency department visits per 1000 long-stay resident days,Processing Date
0,NATION,8.1,4.4,8.4,4.6,8.1,4.3,76.1,2.19,0.88,...,19.669741,8.328548,95.825016,73.115406,78.789720,22.866740,10.489013,1.446098,0.794065,2022-01-01
1,AK,8.7,7.7,12.8,4.7,12.1,9.1,35.3,4.73,0.67,...,17.222425,6.497252,97.293507,76.823628,81.053333,15.731127,10.859274,0.711204,0.814816,2022-01-01
2,AL,2.9,3.0,3.0,3.3,4.0,3.5,87.6,2.20,0.90,...,24.154599,9.364816,93.862901,70.674791,79.280988,22.683458,11.068544,1.650541,0.810642,2022-01-01
3,AR,8.8,1.9,11.2,2.2,7.7,2.2,67.8,2.45,1.04,...,21.732916,8.835729,97.179128,77.538899,77.129412,24.420896,12.353823,1.888193,1.145363,2022-01-01
4,AZ,7.8,3.1,7.8,3.3,5.3,3.6,72.6,2.23,1.10,...,17.063047,9.175842,96.940522,72.479825,88.446259,23.891657,9.618150,1.284218,0.616814,2022-01-01


In [40]:
quality_mds_df = pd.read_csv('NH_QualityMsr_MDS_Jan2022.csv', low_memory=False)
quality_mds_df.describe()

,Provider Zip Code,Measure Code,Q1 Measure Score,Footnote for Q1 Measure Score,Q2 Measure Score,Footnote for Q2 Measure Score,Q3 Measure Score,Footnote for Q3 Measure Score,Q4 Measure Score,Footnote for Q4 Measure Score,Four Quarter Average Score,Footnote for Four Quarter Average Score
count,274284.000000,274284.000000,225444.000000,48840.000000,225668.000000,48616.000000,227888.000000,46396.000000,229504.000000,44780.000000,261466.000000,12818.000000
mean,50311.652776,427.833333,31.429427,9.029484,30.769393,9.028509,30.398275,9.025261,30.375045,9.021103,31.559917,9.037057
std,26831.167658,23.949644,36.889926,0.169161,36.618964,0.166424,36.949114,0.156918,36.884896,0.143730,36.236461,0.188910
min,660.000000,401.000000,0.000000,9.000000,0.000000,9.000000,0.000000,9.000000,0.000000,9.000000,0.000000,9.000000
25%,29841.000000,407.000000,2.298850,9.000000,2.173910,9.000000,2.000000,9.000000,1.990607,9.000000,2.564103,9.000000
50%,48763.000000,417.000000,12.382744,9.000000,11.864410,9.000000,10.627650,9.000000,10.638300,9.000000,12.775330,9.000000
75%,72855.000000,452.000000,62.962960,9.000000,60.606060,9.000000,62.500000,9.000000,62.068970,9.000000,63.999999,9.000000
max,99929.000000,472.000000,100.000000,10.000000,100.000000,10.000000,100.000000,10.000000,100.000000,10.000000,100.000000,10.000000
